# Modelagem de tópicos em textos através do Latent Dirichlet Allocation (LDA) aplicação em python 


## imports 

In [1]:
print('Ola')

Ola


In [1]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

[nltk_data] Downloading package stopwords to /home/renato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = pd.read_csv('./files test/Noticias.csv')

df.head()

,Unnamed: 0,Unnamed: 0.1,TITLE,SUMMARY,text,polaridade
0,0,0,Confira o 2º dia de desfiles da Festlingerie J...,"<img src=""https://s2.glbimg.com/i-uqwYxqUJOApB...",festival lingerie juruaia segue até dia setemb...,Positivo
1,1,1,Queimada consumiu 850 hectares do Parque Estad...,"<img src=""https://s2.glbimg.com/o1U8CVISCkqRNp...",chamas foram controladas depois seis dias caus...,Positivo
2,2,2,Polícia investiga suspeita de mortes por inges...,Três pessoas já morreram e outra está hospital...,três pessoas morreram outra está hospitalizada...,Positivo
3,3,3,Cadela abre buraco em raiz de árvore para ter ...,"<img src=""https://s2.glbimg.com/oZVJrG_qs35bZ8...",todo seis cachorros foram retirados buraco que...,Negativo
4,4,4,PE confirma mais quatro casos e totaliza 22 re...,"<img src=""https://s2.glbimg.com/Gv9miVzhCl5Qtn...",pacientes com casos confirmados nesta terçafei...,Positivo


In [8]:
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])
df.head()

,TITLE,SUMMARY,text,polaridade
0,Confira o 2º dia de desfiles da Festlingerie J...,"<img src=""https://s2.glbimg.com/i-uqwYxqUJOApB...",festival lingerie juruaia segue até dia setemb...,Positivo
1,Queimada consumiu 850 hectares do Parque Estad...,"<img src=""https://s2.glbimg.com/o1U8CVISCkqRNp...",chamas foram controladas depois seis dias caus...,Positivo
2,Polícia investiga suspeita de mortes por inges...,Três pessoas já morreram e outra está hospital...,três pessoas morreram outra está hospitalizada...,Positivo
3,Cadela abre buraco em raiz de árvore para ter ...,"<img src=""https://s2.glbimg.com/oZVJrG_qs35bZ8...",todo seis cachorros foram retirados buraco que...,Negativo
4,PE confirma mais quatro casos e totaliza 22 re...,"<img src=""https://s2.glbimg.com/Gv9miVzhCl5Qtn...",pacientes com casos confirmados nesta terçafei...,Positivo


In [9]:

# remove special characters and digits
def clean_noticia(noticia):
   
    # remove links e alguns pontos
    noticia = re.sub(r'<img src="https\S+', "", noticia)
    noticia = re.sub(r"http\S+", "", noticia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    
    # remove alguns caracteres
    noticia  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",noticia )
    noticia = ' '.join(s for s in noticia.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    noticia = noticia.replace("\n", "")
    return noticia

In [12]:
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'dos'])
clean_noticias = []
for w in range(len(df.SUMMARY)):
  if df['SUMMARY'].iloc[w] == 'nan' or df['SUMMARY'].iloc[w] == '-' or type(df['SUMMARY'].iloc[w]) !=" <class 'str'>":
    noticia = df['TITLE'].iloc[w]
  else:
    noticia = df['SUMMARY'].iloc[w]
  #print(type(noticia))
  noticia = clean_noticia(noticia)
  #print(noticia)  
  clean_noticias.append(noticia)

#remover colunas repetidas
print(len(clean_noticias))
clean_noticias[:10]


15612


['festival lingerie juruaia segue até dia setembro forma totalmente online com desfiles lançamentos coleção primaveraverão confira dia desfiles festlingerie começou festival lingerie juruaia festlingerie por causa pandemia novo coronavírus assim como felinju evento totalmente online com desfiles exposição produtos para apresentar peças coleção primaveraverão mais lojas festival que funciona como uma vitrine setor para natal transmitido através internet site evento pelas redes sociais confira toda cobertura festlingerie nos dias setembro festlingerie realiza uma rodada negócios para novos acordos comerciais promovida pelo serviço brasileiro apoio micro pequenas empresas sebrae minas gerais festlingerie evento promovido pela aciju parceria com sebrae minas está disponível site festlingeriejuruaiacombr confira acima íntegra segundo desfile realizado vivo pelo festlingerie nesta terçafeira confecções lançam novidades lingeries para primaveraverão juruaia reprodução festilingerie veja mais 

In [13]:
# *****
clean_noticias = list(df['text'])
clean_noticias[:10]

['festival lingerie juruaia segue até dia setembro forma totalmente online com desfiles lançamentos coleção primaveraverão confira dia desfiles festlingerie começou festival lingerie juruaia festlingerie por causa pandemia novo coronavírus assim como felinju evento totalmente online com desfiles exposição produtos para apresentar peças coleção primaveraverão mais lojas festival que funciona como uma vitrine setor para natal transmitido através internet site evento pelas redes sociais confira toda cobertura festlingerie nos dias setembro festlingerie realiza uma rodada negócios para novos acordos comerciais promovida pelo serviço brasileiro apoio micro pequenas empresas sebrae minas gerais festlingerie evento promovido pela aciju parceria com sebrae minas está disponível site festlingeriejuruaiacombr confira acima íntegra segundo desfile realizado vivo pelo festlingerie nesta terçafeira confecções lançam novidades lingeries para primaveraverão juruaia reprodução festilingerie veja mais 

###  frequências de palavras TFs

In [14]:
# COUNT vectorizer
tf_vectorizer = CountVectorizer(
        min_df = 30,
        max_df = 0.5,
        max_features = 10000,
        stop_words = stop_words, 
        ngram_range = (1,2)
  )

#transform
vec_text = tf_vectorizer.fit_transform(clean_noticias)

#returns a list of words.
words = tf_vectorizer.get_feature_names()
#print(clean_tweets)

print(len(words))

4379


In [15]:
from sklearn.decomposition import LatentDirichletAllocation

In [16]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = df[['TITLE', 'SUMMARY']].iloc[d]
          print(f'{doc_data[0]} -  \t{W[d, topic_idx]:.2f}')

In [20]:
lda = LatentDirichletAllocation(n_components=15, 
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text)

print("done in %0.3fs." % (time() - t0))

done in 50.795s.


In [21]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(15612, 15)
Matriz tópicos-termos:(15, 4379)


In [22]:
display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               df,
               15, 
               10)


--
Topic #1: 
HREF, TARGET _BLANK, _BLANK, TARGET, HREF TARGET, CADASTRO, INTERNET, VAGAS, POST, DIA, ÚNICO, SER, HORAS, JORNAL, CADASTRO ÚNICO
Morre Gerson King Combo, estrela da soul music brasileira -  	0.91
Campanhas têm estrutura para punir abuso nas redes -  	0.88
Luz no fim do túnel: reciclagem em destaque na capital federal -  	0.87
Restaurante lança parrillada argentina às terças -  	0.87
Cadela amamenta filhotes de gato que foram resgatados após morte da mãe -  	0.87
Terminam hoje inscrições em pós-graduação gratuita para professores -  	0.86
Edição Digital 29-09-2020 -  	0.84
Charge do dia -  	0.84
Edição Digital 24-09-2020 -  	0.84
‘Paciente de Berlim’, primeiro homem curado do HIV, enfrenta câncer terminal -  	0.84

--
Topic #2: 
APARECEU, PRIMEIRO, SOBRE, POST, PRESIDENTE, BRASIL, BOLSONARO, DISSE, APARECEU PRIMEIRO, MUNDO, DIGITAL, DIÁRIO, ESTADOS, WHATSAPP, DIÁRIO DIGITAL
Biden conquista apoio de viúva de McCain, e Trump faz comício -  	0.92
Bolsonaro erra ao falar de 

Paraná registra mais de 175 mil casos confirmados de coronavírus -  	0.97
Paraná registra 63 mortes no boletim desta quarta-feira (30) -  	0.97

--
Topic #15: 
ANOS, CASA, APÓS, REDES, SOCIAIS, FAMÍLIA, SER, REDES SOCIAIS, MULHERES, MÃE, DISSE, DOIS, ONDE, FILHO, VIDA
Rafael Cardoso, mulher e filhos estão com covid-19 -  	0.92
A Hora da Venenosa: Instagram remove foto imprópria de Zé Neto -  	0.89
VÍDEO | Bebê dorme em cima de violão enquanto pai canta e viraliza nas redes sociais -  	0.88
PR: mulher mata filho de três anos e joga corpo em matagal -  	0.88
Mulher mata o marido com tiro na cabeça após discussão -  	0.88
Vídeo mostra bandidos invadindo casa para matar morador em Várzea Grande -  	0.88
Grávida pula de altura de quase 4 metros para fugir de incêndio; mulher teve que jogar a filha pela janela para salvá-la -  	0.87
Mãe descobre 'parte sensível' de tigre sedado a 240 km/h. Entenda! -  	0.87
Macabro! Mãe descobre que caixão do filho estava vazio por 42 anos -  	0.87
Após 1h30

## salvando topicos em um df
https://medium.com/data-hackers/como-categorizar-textos-usando-o-lda-c47c5f6929a

In [23]:
# Listando os topicos e palavras
topics = {
    'topic_1': ['HREF', 'TARGET _BLANK', '_BLANK', 'TARGET', 'HREF TARGET', 'CADASTRO, INTERNET', 'VAGAS', 'POST', 'DIA', 'ÚNICO', 'SER', 'HORAS', 'JORNAL', 'CADASTRO ÚNICO'],
    'topic_2': ['APARECEU', 'PRIMEIRO', 'SOBRE', 'POST', 'PRESIDENTE', 'BRASIL', 'BOLSONARO', 'DISSE', 'APARECEU PRIMEIRO', 'MUNDO', 'DIGITAL', 'DIÁRIO', 'ESTADOS', 'WHATSAPP', 'DIÁRIO DIGITAL'],
    'topic_3': ['NESTE', 'SÁBADO', 'DOMINGO', 'GRANDE', 'CAMPO', 'NESTA', 'ANIMAIS', 'DIVULGAÇÃO', 'MEIO', 'AMBIENTAL', 'QUARTAFEIRA', 'PAULO', 'NESTE DOMINGO', 'SANTO', 'ANIMAL'],
    'topic_4': ['POLÍCIA', 'HOMEM', 'CIVIL', 'CRIME', 'VÍTIMA', 'POLICIAIS', 'SUSPEITO', 'POLÍCIA CIVIL', 'MILITAR', 'PRESO', 'SEGUNDO', 'OPERAÇÃO', 'ANOS', 'POLÍCIA MILITAR', 'DOIS'],
    'topic_5': ['NESTA', 'RIO', 'DIA', 'QUINTAFEIRA', 'SEMANA', 'SUL', 'MINAS', 'SETEMBRO', 'NESTA QUINTAFEIRA', 'JANEIRO', 'SEXTAFEIRA', 'CONFIRA', 'QUARTAFEIRA', 'REGIÃO', 'RIO JANEIRO'],
    'topic_6': ['APARECEU', 'INSS', 'APOSENTADORIA', 'PODE', 'BENEFÍCIO', 'FGTS', 'SER', 'DIREITO', 'SAQUE', 'RECEBER', 'PAGAMENTO', 'CONCESSÃO', 'VALOR', 'PENSÃO', 'PODE SER'],
    'topic_7': ['REL', 'HREF REL', 'REL NOFOLLOW', 'NOFOLLOW', 'HREF', 'PRIMEIRO HREF', 'PRIMEIRO', 'APARECEU PRIMEIRO', 'NOTÍCIA', 'NOTÍCIA HREF', 'WWWMIXVALECOMBR', 'MIXVALE', 'MIXVALE WWWMIXVALECOMBR', 'NOFOLLOW MIXVALE', 'INSS APARECEU'],
    'topic_8': ['INSS', 'SEGURO', 'NACIONAL', 'SOCIAL', 'SEGURADOS', 'INSTITUTO', 'INSTITUTO NACIONAL', 'ATENDIMENTO', 'NACIONAL SEGURO', 'SEGURO SOCIAL', 'REGRAS', 'AGÊNCIAS', 'PERÍCIA', 'PANDEMIA', 'DESEMPREGO'],
    'topic_9': ['CANDIDATOS', 'TRIBUNAL', 'ELEITORAL', 'ELEIÇÕES', 'PREFEITO', 'CANDIDATO', 'JUSTIÇA', 'PREFEITURA', 'CÂMARA', 'PAULO', 'DECISÃO', 'CAMPANHA', 'FEDERAL', 'BENS', 'SUPERIOR'],
    'topic_10': ['AUXÍLIO', 'EMERGENCIAL', 'APARECEU', 'AUXÍLIO EMERGENCIAL', 'FAMÍLIA', 'BOLSA', 'BOLSA FAMÍLIA', 'GOVERNO', 'MILHÕES', 'PROGRAMA', 'PAGAMENTO', 'CAIXA', 'PARCELAS', 'RENDA', 'BENEFICIÁRIOS'],
    'topic_11': ['PROJETO', 'OUTUBRO', 'ENSINO', 'VIDA', 'EDUCAÇÃO', 'GRUPO', 'SAÚDE', 'SOBRE', 'PROVA', 'DETRAN', 'AÇÃO', 'FAZ', 'PROCESSO', 'PRODUÇÃO', 'HORIZONTE'],
    'topic_12': ['REGIÃO', 'DESTA', 'CARRO', 'LOCAL', 'ACIDENTE', 'SEGUNDO', 'MOTORISTA', 'BOMBEIROS', 'VEÍCULO', 'CORPO', 'VEJA', 'NOITE', 'MANHÃ', 'INCÊNDIO', 'RUA'],
    'topic_13': ['ANO', 'PANDEMIA', 'MIL', 'SEGUNDO', 'MÊS', 'AGOSTO', 'MESES', 'MAIOR', 'EMPRESA', 'PESSOAS', 'DESDE', 'AINDA', 'DURANTE', 'BRASIL', 'DESTE'],
    'topic_14': ['CASOS', 'CORONAVÍRUS', 'MORTES', 'SAÚDE', 'DOENÇA', 'ESTADO', 'NOTÍCIAS', 'VEJA', 'REGIÃO', 'PACIENTES', 'ÓBITOS', 'CONFIRMADOS', 'NÚMERO', 'ASSISTA', 'NESTA'],
    'topic_15': ['ANOS', 'CASA', 'APÓS', 'REDES', 'SOCIAIS', 'FAMÍLIA', 'SER', 'REDES SOCIAIS', 'MULHERES', 'MÃE', 'DISSE', 'DOIS', 'ONDE', 'FILHO', 'VIDA']
}

In [32]:
# Função que pega uma lista de palavras e um texto como entrada e conta quantas vezes as palavras da lista aparecem no texto
def count_words(words, text):
    counter = 0
    for word in words:
        for w in text.split():
            if word == w:
                counter+= 1
    return counter

# Função para classificar cada resumo em um tópico, verificando o tópico que contém mais palavras.
# Para cada resumo, cria um dicionário {topic: qtd words da lista} e depois verifica qual é a chave no dicionário com o maior número, que será o número do tópico a ser adicionado a uma lista e devolvido.

def assign_topic(topic, text):
    assign = []
    for row in text:
        topic_counts = {}
        for key, value in topic.items():
            v = [x.lower() for x in value]
            topic_counts[key] = count_words(v, row) #use function previously created
        assign.append(max(topic_counts, key=lambda k: topic_counts[k]))
    return assign

In [33]:
# A lista de tópicos que irão no dataframe
topics_list = assign_topic(topics, df.text)

# Adicionando a coluna de tópicos ao dataframe
df['topics'] = topics_list
df.sample(25)

,TITLE,SUMMARY,text,polaridade,topics
14932,Jesus voltou; deu ruim,"Eis que, conforme amplamente anunciado, Jesus ...",eis que conforme amplamente anunciado jesus vo...,Negativo,topic_1
7935,Homem esfaqueia desafeto e acaba preso ao proc...,NaN,homem esfaqueia desafeto acaba preso procurar ...,Positivo,topic_4
11455,JR 15 min #45: Pandemia faz com que mercado de...,O mercado de games cresce todos os anos no Bra...,mercado games cresce todos anos brasil mas ago...,Positivo,topic_2
8253,Brasil registra 869 novas mortes por Covid-19 ...,SÃO PAULO (Reuters) – O Brasil registrou neste...,são paulo reuters brasil registrou neste sábad...,Positivo,topic_14
14145,Confira dez filmes do É Tudo Verdade para viaj...,"Tudo estava pronto para a <a href=""https://www...",tudo estava pronto para href anos tudo verdade...,Positivo,topic_13
15259,Cerca de 12 mil alunos trocam escola particula...,Cerca de 12 mil alunos foram transferidos de e...,cerca mil alunos foram transferidos escolas pa...,Positivo,topic_13
8986,BC da China diz que vai manter liquidez ampla ...,PEQUIM (Reuters) – O banco central da China af...,pequim reuters banco central china afirmou nes...,Positivo,topic_15
13017,Um em cada oito brasileiros sofreu tentativas ...,Segundo relatório de um dos maiores escritório...,segundo relatório dos maiores escritórios segu...,Positivo,topic_13
7522,Morador de Toledo é preso com drogas,O homem de 31 anos foi preso com quase meia to...,homem anos foi preso com quase meia tonelada m...,Positivo,topic_4
10714,Auxílio Emergencial: Confira o calendário de p...,<p>O calendário de depósitos do auxílio emerge...,calendário depósitos auxílio emergencial para ...,Positivo,topic_10


In [37]:
def get_termos(topic):
    return topics[str(topic)]
df['termos'] = np.array([get_termos(topic) for topic in df['topics'].values.astype('U')])
df.sample(25)

<ipython-input-37-a23bcd943b72>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  df['termos'] = np.array([get_termos(topic) for topic in df['topics'].values.astype('U')])


,TITLE,SUMMARY,text,polaridade,topics,termos
6001,AGU suspende promoção de 606 servidores ao top...,"<a href=""https://jornaldebrasilia.com.br/polit...",href target _blank agu revogou decisão setembr...,Negativo,topic_1,"[HREF, TARGET _BLANK, _BLANK, TARGET, HREF TAR..."
12693,Família de vereador morto há quatro anos esper...,"Há quase quatro anos, um vereador da cidade de...",quase quatro anos vereador cidade santa helena...,Positivo,topic_4,"[POLÍCIA, HOMEM, CIVIL, CRIME, VÍTIMA, POLICIA..."
1201,Mulher é esfaqueada pelo namorado na frente do...,"Segundo a Polícia Militar, a agressão começou ...",segundo polícia militar agressão começou por c...,Positivo,topic_4,"[POLÍCIA, HOMEM, CIVIL, CRIME, VÍTIMA, POLICIA..."
3707,De milkshake de graça a versões caninas: casas...,"<a href=""https://jornaldebrasilia.com.br/gastr...",href target _blank data celebrada nesta quarta...,Negativo,topic_1,"[HREF, TARGET _BLANK, _BLANK, TARGET, HREF TAR..."
2066,Setembro termina com temperatura máxima de 39°...,"<img src=""https://s2.glbimg.com/i7kNm_BR3Dx7W0...",defesa civil estadual emitiu alerta para forte...,Positivo,topic_5,"[NESTA, RIO, DIA, QUINTAFEIRA, SEMANA, SUL, MI..."
14748,Joice Hasselmann registra candidatura sem plan...,A deputada federal Joice Hasselmann (PSL-SP) r...,deputada federal joice hasselmann pslsp regist...,Positivo,topic_9,"[CANDIDATOS, TRIBUNAL, ELEITORAL, ELEIÇÕES, PR..."
10747,Fumaça de incêndios no Pantanal volta a encobr...,NaN,fumaça incêndios pantanal volta encobrir cuiabá,Negativo,topic_1,"[HREF, TARGET _BLANK, _BLANK, TARGET, HREF TAR..."
8055,EUA impõem restrições sobre exportações para f...,NEGOCIOS-EUA-SMIC:EUA impõem restrições sobre ...,negocioseuasmiceua impõem restrições sobre exp...,Positivo,topic_2,"[APARECEU, PRIMEIRO, SOBRE, POST, PRESIDENTE, ..."
11276,"Cruzeiro, Asa Norte e Samambaia ficarão sem en...","<a href=""https://jornaldebrasilia.com.br/cidad...",href target _blank regiões terão fornecimento ...,Positivo,topic_1,"[HREF, TARGET _BLANK, _BLANK, TARGET, HREF TAR..."
9704,Vídeo: tanque de vinho explode e derrama 50 mi...,NaN,vídeo tanque vinho explode derrama mil litros ...,Negativo,topic_13,"[ANO, PANDEMIA, MIL, SEGUNDO, MÊS, AGOSTO, MES..."


In [38]:
df.to_csv('./files test/Noticias.csv',index=False)

## vizualização

In [13]:
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [14]:
pyLDAvis.sklearn.prepare(lda, vec_text, tf_vectorizer, sort_topics=False, mds = 'tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  12.179681
1       0.648955  71.707100       2        1  10.610272
2      71.019684   9.971006       3        1  11.311983
3       3.785981  33.185135       4        1   9.438297
4     -42.666500  -9.222070       5        1  13.321389
5      -3.621689 -43.526516       6        1   7.295597
6      -3.632402  -4.171708       7        1   8.721656
7     -36.584400  35.804413       8        1   9.670168
8      44.391380  50.615383       9        1   8.950345
9      40.235809 -31.591665      10        1   8.500610, topic_info=                  Term         Freq        Total Category  logprob  loglift
338               inss  1575.000000  1575.000000  Default  30.0000  30.0000
224                diz   744.000000   744.000000  Default  29.0000  29.0000
65             auxílio   630.000000   630.000000  Default  28.0000  28.0000
42                após   752.000000   752.000000  Default  27.0000  27.0000
659              sobre   463.000000   463.000000  Default  26.0000  26.0000
..                 ...          ...          ...      ...      ...      ...
380              marca    50.681631    51.597733  Topic10  -4.6125   2.4471
439  nesta quartafeira    50.070393    50.986477  Topic10  -4.6246   2.4469
42                após   167.378339   752.057592  Topic10  -3.4178   0.9625
646                ser   129.718149   487.949860  Topic10  -3.6727   1.1402
233            durante    59.571157   246.726209  Topic10  -4.4509   1.0439

[362 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         2  0.988118      abre
3         5  0.983632    acesso
4         7  0.992449  acidente
6         6  0.983284    acordo
11        7  0.991672    agosto
...     ...       ...       ...
743       7  0.991772      zona
744      10  0.977002      água
746       6  0.983887   íntegra
748       7  0.983600    ônibus
751       2  0.994450     único

[345 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])